In [20]:
import tensorflow as tf
import numpy as np

In [21]:
# Flatten
model = tf.keras.models.Sequential()
# Add layers
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

In [22]:
# The loss method
# je crée mon propre loss plutot que d'utiliser la variable loss au sein de model.compil
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
# The optimize
# sélectionne la méthode de gradient à utiliser pour réduire l'erreur
optimizer = tf.keras.optimizers.Adam()
# This metrics is used to track the progress of the training loss during the training
# pour observer l'évolution de mon erreur et enregistrer les erreurs relevés à chaque boucles au sein de mon epoch
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [23]:
@tf.function
def train_step(image, targets):
    # avec with on peut capturer tout ce qui se passe pour calculer le gradient
    with tf.GradientTape() as tape:
        # Make a prediction
        predictions = model(image)
        # Get the error/loss using the loss_object previously defined
        loss = loss_object(targets, predictions)
    # Compute the gradient which respect to the loss
    # mes gradients seront mes poids modifiable, ici j'ai 30 entrées dans images analysé pas 256 neurones sur le premier layer
    # puis 256 valeurs analysées par 128 neurones sur mon deuxième layer et enfin 128 valeurs analysées par 2 neurones 
    # avant la sortie de la valeur
    gradients = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    # pour lier chaques gradient avec les variables du modèle
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)

In [24]:
for epoch in range(0, 10):
    for _ in range(0, 10):
        # Create fake inputs with two class
        # Create fake inputs
        # 1 matrice avec 2 vecteurs de 30 valeurs, premier vecteur avce des valeurs à -1 et deuxième à 1
        inputs = np.zeros((2, 30))
        inputs[0] -= 1
        inputs[1] = 1
        # Create fake targets
        # 1 matrice avec 2 vecteurs de 1 valeur, premier vecteur avce des valeurs à 0 et deuxième à 1
        targets = np.zeros((2, 1))
        targets[0] = 0
        targets[1] = 1
        # Train the model
        train_step(inputs, targets)
    print("Loss: " ,train_loss.result())
    train_loss.reset_states()


Loss:  tf.Tensor(0.20883875, shape=(), dtype=float32)
Loss:  tf.Tensor(0.005423023, shape=(), dtype=float32)
Loss:  tf.Tensor(0.0008223917, shape=(), dtype=float32)
Loss:  tf.Tensor(0.00031253826, shape=(), dtype=float32)
Loss:  tf.Tensor(0.00019441349, shape=(), dtype=float32)
Loss:  tf.Tensor(0.0001514499, shape=(), dtype=float32)
Loss:  tf.Tensor(0.00012969883, shape=(), dtype=float32)
Loss:  tf.Tensor(0.000115467854, shape=(), dtype=float32)
Loss:  tf.Tensor(0.00010439511, shape=(), dtype=float32)
Loss:  tf.Tensor(9.4990886e-05, shape=(), dtype=float32)


In [31]:
# utilisation de notre modèle entrainé plus haut pour prédire si la valeur est 1 ou non
# numpy=array([[2.2915769e-05, 9.9997711e-01]] on voit que la deuxième valeur est plus haute, la prédiction est donc sur 1

input_ = np.zeros((1, 30)) + 1
model(input_)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[2.2915769e-05, 9.9997711e-01]], dtype=float32)>